In [ ]:
# 사용 라이브러리
!pip install --target=$my_path category_encoders
!pip install --target=$my_path catboost

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings, random
warnings.filterwarnings(action='ignore')

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from category_encoders.ordinal import OrdinalEncoder
from category_encoders import OneHotEncoder
from sklearn.model_selection import StratifiedKFold

from sklearn.cluster import KMeans
from catboost import CatBoostClassifier, Pool

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
secom_train = pd.read_csv('train.csv')
train = secom_train.copy()
secom_test = pd.read_csv('test.csv')
test = secom_test.copy()

In [ ]:
train = train.dropna(subset=['HEAR_LEFT', 'HEAR_RIGHT'])

In [ ]:
train[['HEAR_LEFT', 'HEAR_RIGHT']].describe()

,HEAR_LEFT,HEAR_RIGHT
count,99988.000000,99988.000000
mean,1.034034,1.032674
std,0.181318,0.177783
min,1.000000,1.000000
25%,1.000000,1.000000
50%,1.000000,1.000000
75%,1.000000,1.000000
max,2.000000,2.000000


In [ ]:
# 문자형 0과 1로 바꿈
train = train.replace('Y', 1)
train = train.replace('N', 0)

test = test.replace('Y', 1)
test = test.replace('N', 0)

In [ ]:
# 결측치 예측 함수
imputer_mice = IterativeImputer(random_state=83)
train = imputer_mice.fit_transform(train)

imputer_mice = IterativeImputer(random_state=83)
test = imputer_mice.fit_transform(test)

In [ ]:
# 위 함수로 바뀐거 다시 데이터프레임으로 고침
train = pd.DataFrame(train)
train.columns = ['ID', 'SEX', 'AGE', 'SIDO', 'HEIGHT', 'WEIGHT', 'WAIST', 'SIGHT_LEFT', 'SIGHT_RIGHT', 'HEAR_LEFT', 'HEAR_RIGHT', 'BP_HIGH', 'BP_LWST', 'BLDS', 'TOT_CHOLE', 'TRIGLYCERIDE', 'HDL_CHOLE', 'LDL_CHOLE', 'HMG', 'OLIG_PROTE', 'CREATININE', 'SGOT_AST', 'SGOT_ALT', 'GAMMA_GTP', 'DRK_YN', 'HCHK_OE_INSPEC_YN', 'CRS_YN', 'TTR_YN', 'SMK_STAT']

test = pd.DataFrame(test)
test.columns = ['ID', 'SEX', 'AGE', 'SIDO', 'HEIGHT', 'WEIGHT', 'WAIST', 'SIGHT_LEFT', 'SIGHT_RIGHT', 'HEAR_LEFT', 'HEAR_RIGHT', 'BP_HIGH', 'BP_LWST', 'BLDS', 'TOT_CHOLE', 'TRIGLYCERIDE', 'HDL_CHOLE', 'LDL_CHOLE', 'HMG', 'OLIG_PROTE', 'CREATININE', 'SGOT_AST', 'SGOT_ALT', 'GAMMA_GTP', 'DRK_YN', 'HCHK_OE_INSPEC_YN', 'CRS_YN', 'TTR_YN']

In [ ]:
# 이상치 처리(int값만 없애기 가능, 결측치 있으면 불가)
train = train[(train['AGE'] <= 97.5)]
train = train[(train['HEIGHT'] <= 192.5)]
train = train[(train['WEIGHT'] <= 92.5)]
train = train[(train['WAIST'] <= 107.5)]
train = train[(train['SIGHT_LEFT'] <= 2.5)]
train = train[(train['SIGHT_RIGHT'] <= 2.5)]
train = train[(train['BP_HIGH'] <= 159.5)]
train = train[(train['BP_LWST'] <= 100.0)]
train = train[(train['BLDS'] <= 131.5)]
train = train[(train['TOT_CHOLE'] <= 220.0)]
train = train[(train['TRIGLYCERIDE'] <= 250.0)]
train = train[(train['HDL_CHOLE'] <= 85.0)]
train = train[(train['LDL_CHOLE'] <= 133.5)]
train = train[(train['HMG'] <= 18.0)]
train = train[(train['CREATININE'] <= 2.5)]
train = train[(train['SGOT_AST'] <= 44.0)]
train = train[(train['SGOT_ALT'] <= 52.5)]
train = train[(train['GAMMA_GTP'] <= 78.5)]
train = train.reset_index(drop=True)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66832 entries, 0 to 66831
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 66832 non-null  float64
 1   SEX                66832 non-null  float64
 2   AGE                66832 non-null  float64
 3   SIDO               66832 non-null  float64
 4   HEIGHT             66832 non-null  float64
 5   WEIGHT             66832 non-null  float64
 6   WAIST              66832 non-null  float64
 7   SIGHT_LEFT         66832 non-null  float64
 8   SIGHT_RIGHT        66832 non-null  float64
 9   HEAR_LEFT          66832 non-null  float64
 10  HEAR_RIGHT         66832 non-null  float64
 11  BP_HIGH            66832 non-null  float64
 12  BP_LWST            66832 non-null  float64
 13  BLDS               66832 non-null  float64
 14  TOT_CHOLE          66832 non-null  float64
 15  TRIGLYCERIDE       66832 non-null  float64
 16  HDL_CHOLE          668

# Feature engineering

In [ ]:
# 변수 삭제
train = train.drop(['ID', 'SIDO', 'HCHK_OE_INSPEC_YN', 'CRS_YN', 'TTR_YN'], axis=1)

In [ ]:
test = test.drop(['ID', 'SIDO', 'HCHK_OE_INSPEC_YN', 'CRS_YN', 'TTR_YN'], axis=1)

In [ ]:
train.columns

Index(['SEX', 'AGE', 'HEIGHT', 'WEIGHT', 'WAIST', 'SIGHT_LEFT', 'SIGHT_RIGHT',
       'HEAR_LEFT', 'HEAR_RIGHT', 'BP_HIGH', 'BP_LWST', 'BLDS', 'TOT_CHOLE',
       'TRIGLYCERIDE', 'HDL_CHOLE', 'LDL_CHOLE', 'HMG', 'OLIG_PROTE',
       'CREATININE', 'SGOT_AST', 'SGOT_ALT', 'GAMMA_GTP', 'DRK_YN',
       'SMK_STAT'],
      dtype='object')

In [ ]:
# 중복되는 변수들 새로운 파생변수로 만들기
train['BP_AVER'] = (train['BP_HIGH'] + train['BP_LWST'])/2
train = train.drop(['BP_HIGH', 'BP_LWST'], axis = 1)

train['SGOT_AVER'] = (train['SGOT_AST'] + train['SGOT_ALT'])/2
train = train.drop(['SGOT_AST', 'SGOT_ALT'], axis = 1)

test['BP_AVER'] = (test['BP_HIGH'] + test['BP_LWST'])/2
test = test.drop(['BP_HIGH', 'BP_LWST'], axis = 1)

test['SGOT_AVER'] = (test['SGOT_AST'] + test['SGOT_ALT'])/2
test = test.drop(['SGOT_AST', 'SGOT_ALT'], axis = 1)

In [ ]:
# 시력 변수 하나로
train['SIGHT_AVER'] = (train['SIGHT_LEFT'] + train['SIGHT_RIGHT'])/2
train = train.drop(['SIGHT_LEFT', 'SIGHT_RIGHT'], axis = 1)

In [ ]:
test['SIGHT_AVER'] = (test['SIGHT_LEFT'] + test['SIGHT_RIGHT'])/2
test = test.drop(['SIGHT_LEFT', 'SIGHT_RIGHT'], axis = 1)

In [ ]:
# 범주형 청력 변수 하나로 만들기 -> 1 정상, 0 비정상
train.loc[train['HEAR_LEFT'] == 2, 'HEAR_LEFT'] = 0
train.loc[train['HEAR_RIGHT'] == 2, 'HEAR_RIGHT'] = 0

train['HEAR'] = train['HEAR_LEFT']*train['HEAR_RIGHT']
train = train.drop(['HEAR_LEFT', 'HEAR_RIGHT'], axis = 1)

In [ ]:
test.loc[test['HEAR_LEFT'] == 2, 'HEAR_LEFT'] = 0
test.loc[test['HEAR_RIGHT'] == 2, 'HEAR_RIGHT'] = 0

test['HEAR'] = test['HEAR_LEFT']*test['HEAR_RIGHT']
test = test.drop(['HEAR_LEFT', 'HEAR_RIGHT'], axis = 1)

In [ ]:
# Y값 중 2 -> 1로 바꾸기
train.loc[train['SMK_STAT'] == 2, 'SMK_STAT'] = 1

In [ ]:
train.columns

Index(['SEX', 'AGE', 'HEIGHT', 'WEIGHT', 'WAIST', 'BLDS', 'TOT_CHOLE',
       'TRIGLYCERIDE', 'HDL_CHOLE', 'LDL_CHOLE', 'HMG', 'OLIG_PROTE',
       'CREATININE', 'GAMMA_GTP', 'DRK_YN', 'SMK_STAT', 'BP_AVER', 'SGOT_AVER',
       'SIGHT_AVER', 'HEAR'],
      dtype='object')

In [ ]:
train[['SEX', 'DRK_YN', 'SMK_STAT', 'HEAR']] = train[['SEX', 'DRK_YN', 'SMK_STAT', 'HEAR']].astype(dtype='object')

In [ ]:
test[['SEX', 'DRK_YN', 'HEAR']] = test[['SEX', 'DRK_YN', 'HEAR']].astype(dtype='object')

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66832 entries, 0 to 66831
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   SEX           66832 non-null  object 
 1   AGE           66832 non-null  float64
 2   HEIGHT        66832 non-null  float64
 3   WEIGHT        66832 non-null  float64
 4   WAIST         66832 non-null  float64
 5   BLDS          66832 non-null  float64
 6   TOT_CHOLE     66832 non-null  float64
 7   TRIGLYCERIDE  66832 non-null  float64
 8   HDL_CHOLE     66832 non-null  float64
 9   LDL_CHOLE     66832 non-null  float64
 10  HMG           66832 non-null  float64
 11  OLIG_PROTE    66832 non-null  float64
 12  CREATININE    66832 non-null  float64
 13  GAMMA_GTP     66832 non-null  float64
 14  DRK_YN        66832 non-null  object 
 15  SMK_STAT      66832 non-null  object 
 16  BP_AVER       66832 non-null  float64
 17  SGOT_AVER     66832 non-null  float64
 18  SIGHT_AVER    66832 non-nu

In [ ]:
numerical_feats = train.dtypes[train.dtypes != "object"].index.tolist()
categorical_feats = train.dtypes[train.dtypes == "object"].index.tolist()

In [ ]:
numerical_feats2 = test.dtypes[test.dtypes != "object"].index.tolist()
categorical_feats2 = test.dtypes[test.dtypes == "object"].index.tolist()

In [ ]:
categorical_feats

['SEX', 'DRK_YN', 'SMK_STAT', 'HEAR']

In [ ]:
numerical_feats

['AGE',
 'HEIGHT',
 'WEIGHT',
 'WAIST',
 'BLDS',
 'TOT_CHOLE',
 'TRIGLYCERIDE',
 'HDL_CHOLE',
 'LDL_CHOLE',
 'HMG',
 'OLIG_PROTE',
 'CREATININE',
 'GAMMA_GTP',
 'BP_AVER',
 'SGOT_AVER',
 'SIGHT_AVER']

In [ ]:
train[categorical_feats]

,SEX,DRK_YN,SMK_STAT,HEAR
0,2.0,1.0,0.0,1.0
1,1.0,0.0,1.0,1.0
2,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0
4,2.0,1.0,0.0,1.0
...,...,...,...,...
66827,2.0,1.0,1.0,1.0
66828,1.0,0.0,0.0,1.0
66829,1.0,1.0,1.0,1.0
66830,2.0,0.0,0.0,1.0


In [ ]:
# OrdinalEncoder으로 범주형 변수들 바꾸기
encoder = OrdinalEncoder(categorical_feats)
train[categorical_feats] = encoder.fit_transform(train[categorical_feats])

In [ ]:
encoder = OrdinalEncoder(categorical_feats2)
test[categorical_feats2] = encoder.fit_transform(test[categorical_feats2])

In [ ]:
train.columns

,SEX,AGE,HEIGHT,WEIGHT,WAIST,BLDS,TOT_CHOLE,TRIGLYCERIDE,HDL_CHOLE,LDL_CHOLE,HMG,OLIG_PROTE,CREATININE,GAMMA_GTP,DRK_YN,SMK_STAT,BP_AVER,SGOT_AVER,SIGHT_AVER,HEAR
0,1,-0.822821,-1.297188,0.308852,0.886744,0.295177,1.225667,1.987709,-0.411876,0.594341,0.921521,-0.205101,0.291292,1.460628,1,1,-0.395535,1.590796,0.780725,1
1,2,0.528096,0.344033,0.755003,0.041304,1.851086,0.144682,0.831741,-0.897345,0.292859,0.790255,-0.205101,1.290806,0.417463,2,2,0.710720,0.430586,-0.492343,1
2,2,-0.485092,0.344033,-0.137299,-0.514907,-0.137020,-0.583610,-1.234767,-0.699192,0.476575,0.855888,-0.205101,0.791049,-0.277980,1,2,1.095505,0.575612,0.303324,1
3,2,0.528096,-0.203041,0.308852,0.375030,-0.742096,0.237411,0.595404,-0.482463,0.247836,0.987153,-0.205101,1.290806,1.947438,1,2,-0.299338,0.285560,-0.492343,1
4,1,0.190367,-1.297188,-0.137299,0.185919,-0.050581,0.371644,0.640819,0.671155,-0.288247,-1.375620,-0.205101,-0.708222,0.278375,1,1,2.057465,-0.367058,-0.810610,1


In [ ]:
# 수치형 변수 스케일링
scaler = StandardScaler()
train[numerical_feats] = scaler.fit_transform(train[numerical_feats])

In [ ]:
scaler = StandardScaler()
test[numerical_feats2] = scaler.fit_transform(test[numerical_feats2])

In [ ]:
# 1 비흡연자, 2흡연자
train['SMK_STAT'].value_counts()

1    42717
2    24115
Name: SMK_STAT, dtype: int64

In [ ]:
train = pd.DataFrame(train)
test = pd.DataFrame(test)

# 모델링

In [ ]:
x_data = train.drop('SMK_STAT',axis=1)
y_data = train['SMK_STAT']

In [ ]:
y_data = pd.DataFrame(y_data)
y_data

,SMK_STAT
0,1
1,2
2,2
3,2
4,1
...,...
66827,2
66828,1
66829,2
66830,1


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=777) 

In [ ]:
# 데이터 불균형 smote
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state = 42)
train_over, label_over = smote.fit_resample(x_train, y_train)

In [ ]:
label_over.value_counts()

SMK_STAT
1           29830
2           29830
dtype: int64

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import numpy as np

from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')

# define the parameters grid
param_grid = {'max_depth': np.arange(3, 8),
             'criterion' : ['gini','entropy'],
             'max_leaf_nodes': [5,10,20,50],
             'min_samples_split': [2, 5, 10, 20]}

# create the grid
grid_tree = GridSearchCV(RandomForestClassifier(), param_grid, cv = 5, scoring= 'f1')
# the cv option will be clear in a few cells

#training
grid_tree.fit(x_train, y_train)
#let's see the best estimator
print(grid_tree.best_estimator_)
#with its score
print(np.abs(grid_tree.best_score_))

RandomForestClassifier(max_depth=7, max_leaf_nodes=50, min_samples_split=20)
0.8364898353787271
